<a href="https://colab.research.google.com/github/yuanyangshi/Home-Credit-LGB-Cat-ensemble-_GUO/blob/main/phi_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import threading
import time

def keep_alive():
    while True:
        time.sleep(60)  # 每隔60秒休眠一次

# 创建并启动线程
thread = threading.Thread(target=keep_alive)
thread.start()

In [2]:
from IPython.display import display, HTML

js_code = '''
<script>
function ClickConnect(){
    console.log("模拟点击防止断连");
    document.querySelector("colab-connect-button").click();
}
setInterval(ClickConnect, 60*1000);
</script>
'''
display(HTML(js_code))

In [3]:
# You only need to run this once per machine, even if you stop/restart it
!pip install bitsandbytes
!pip install transformers
!pip install peft
!pip install accelerate
!pip install datasets scipy ipywidgets einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
import wandb
import os

# 设置离线模式和环境变量
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_DISABLE_UI"] = "true"  # 禁用交互式UI提示

# 强制匿名登录，跳过账户选择
wandb.login(anonymous="must")

# 设置项目名称
wandb_project = "viggo-finetune"
if wandb_project:
    os.environ["WANDB_PROJECT"] = wandb_project

# 可选：在初始化时显式设置离线模式
wandb.init(mode="offline")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING Unable to verify login in offline mode.


In [5]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

def merge_and_split_datasets():
    # 加载数据集
    ds_en = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")
    # ds_zh = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh")

    # # 合并两个语言的数据集
    # merged_datasets = {}
    # for split in ds_en.keys():  # 假设两个数据集有相同的 split（如 'train'）
    #     shuffled_en = ds_en[split].shuffle(seed=42).flatten()
    #     shuffled_zh = ds_zh[split].shuffle(seed=42).flatten()

    #     # 使用 concatenate_datasets 进行合并
    #     merged_datasets[split] = concatenate_datasets([shuffled_en, shuffled_zh])


    # 拆分训练集为训练集和验证集
    train_test_valid = ds_en["train"].train_test_split(test_size=0.2)
    train_valid = train_test_valid["train"].train_test_split(test_size=0.1)  # 10% 验证集

    # 重新组织数据集
    final_datasets = DatasetDict({
        "train": train_test_valid["train"],
        "validation": train_valid["test"],
        "test": train_test_valid["test"]
    })

    # 取十分之一的数据集
    sampled_datasets = DatasetDict({
        "train": final_datasets["train"].shuffle().select(range(len(final_datasets["train"]) // 10)),
        "validation": final_datasets["validation"].shuffle().select(range(len(final_datasets["validation"]) // 10)),
        "test": final_datasets["test"].shuffle().select(range(len(final_datasets["test"]) // 10))
    })

    return sampled_datasets

# 运行函数
dataset = merge_and_split_datasets()

# 打印数据集大小
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Question', 'Complex_CoT', 'Response'],
        num_rows: 2029
    })
    validation: Dataset({
        features: ['Question', 'Complex_CoT', 'Response'],
        num_rows: 203
    })
    test: Dataset({
        features: ['Question', 'Complex_CoT', 'Response'],
        num_rows: 507
    })
})


In [6]:
# from datasets import load_dataset

# # 加载数据集
# ds = load_dataset("ticoAg/Chinese-medical-dialogue")

# # 1. 先随机抽取 1% 的数据
# subset = ds["train"].train_test_split(test_size=0.02)["test"]

# # 2. 再划分训练集和测试集（80% 训练，20% 测试）
# train_test = subset.train_test_split(test_size=0.2)

# # 3. 从训练集中划分验证集（10% 训练数据用作验证）
# train_valid = train_test["train"].train_test_split(test_size=0.1)


# # 4. 组合成最终的数据集
# dataset = {
#     "train": train_valid["train"],
#     "validation": train_valid["test"],
#     "test": train_test["test"],
# }

# # 5. 输出数据集大小
# print(f"Train size: {len(dataset['train'])}")
# print(f"Validation size: {len(dataset['validation'])}")
# print(f"Test size: {len(dataset['test'])}")

In [7]:
print(dataset["train"][10])


{'Question': 'An 18-year-old female presents with a hypopigmented patch on both ankles. Based on standard treatment practices, identify a topical medication that is not typically used for treating this condition.', 'Complex_CoT': "Okay, so let's think about this. We have an 18-year-old female with hypopigmented patches on both of her ankles. Hmm, what could cause that? I know that vitiligo is a common condition that leads to patches like that. It's also symmetrical, which matches her symptoms. Another possibility could be something like tinea versicolor, which is fungal and can present as either hyperpigmented or hypopigmented patches. But since we know she has it on both ankles, maybe vitiligo is more likely because it often affects both sides of the body. \n\nNow, let's focus on treatments for vitiligo. I remember that topical corticosteroids are pretty standard; they help to reduce inflammation and might stimulate repigmentation. And then there are calcineurin inhibitors, like tacro

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling

base_model_id = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                             load_in_8bit=True,
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# 5. Tokenization

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_eos_token=True,
    add_bos_token=True,
    use_fast=False, # needed for now, should be fixed soon
)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [10]:
def tokenize(prompt):
    result = tokenizer(prompt)
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
def generate_and_tokenize_prompt(data_point):
    """将数据点转换为适用于 phi-2 训练的文本格式，数据格式为 Question, Complex_CoT, Response"""
    full_prompt = f"""### Question:
{data_point['Question']}

### Complex_CoT:
{data_point['Complex_CoT']}

### Response:
{data_point['Response']}
"""
    return tokenize(full_prompt)

In [12]:
tokenized_train_dataset = dataset['train'].map(generate_and_tokenize_prompt)
tokenized_val_dataset = dataset['validation'].map(generate_and_tokenize_prompt)

Map:   0%|          | 0/2029 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2886 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [13]:
untokenized_text = tokenizer.decode(tokenized_train_dataset[1]['input_ids'])
print(untokenized_text)

<|endoftext|>### Question:
A 12-month-old girl is brought to her pediatrician for a checkup and vaccines. The patient’s mother wants to send her to daycare but is worried about exposure to unvaccinated children and other potential sources of infection. The toddler was born at 39 weeks gestation via spontaneous vaginal delivery. She is up to date on all vaccines. She does not walk yet but stands in place and can say a few words. The toddler drinks formula and eats a mixture of soft vegetables and pureed meals. She has no current medications. On physical exam, the vital signs include: temperature 37.0°C (98.6°F), blood pressure 95/50 mm Hg, pulse 130/min, and respiratory rate 28/min. The patient is alert and responsive. The remainder of the exam is unremarkable. Which of the following is most appropriate for this patient at this visit?
A. Referral for speech pathology
B. MMR vaccine
C. Rotavirus vaccine
D. Meningococcal vaccine
E. Gross motor workup and evaluation

### Complex_CoT:
Let's

In [14]:
# import matplotlib.pyplot as plt

# def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
#     lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
#     lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
#     print(len(lengths))

#     # Plotting the histogram
#     plt.figure(figsize=(10, 6))
#     plt.hist(lengths, bins=20, alpha=0.7, color='blue')
#     plt.xlabel('Length of input_ids')
#     plt.ylabel('Frequency')
#     plt.title('Distribution of Lengths of input_ids')
#     plt.show()

# plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

In [15]:
max_length = 1020 # This was an appropriate max length for my dataset

# redefine the tokenize function and tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
    trust_remote_code=True,
    use_fast=False, # needed for now, should be fixed soon
)
tokenizer.pad_token = tokenizer.eos_token


def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
tokenized_train_dataset = dataset['train'].map(generate_and_tokenize_prompt)
tokenized_val_dataset = dataset['validation'].map(generate_and_tokenize_prompt)

Map:   0%|          | 0/2029 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [17]:
print(tokenized_train_dataset[1]['input_ids'])

[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [18]:
untokenized_text = tokenizer.decode(tokenized_train_dataset[1]['input_ids'])
print(untokenized_text)

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|

In [19]:
# plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

# How does the base model do?
Let's grab a test input (meaning_representation) and desired output (target) pair to see how the base model does on it.

In [20]:
print("Question: " + " ".join(dataset["test"]['Question'][1]))
print("Response: " + " ".join(dataset["test"]['Response'][1]) + "\n")
print("Complex_CoT: " + " ".join(dataset["test"]['Complex_CoT'][1]) + "\n")

Question: A   4 0 - y e a r - o l d   p a d d y   f a r m e r   p r e s e n t e d   w i t h   f e v e r ,   c h i l l s ,   h e a d a c h e   a n d   m y a l g i a s   f o r   2   d a y s .   T h e   p a t i e n t   a l s o   c o m p l a i n e d   o f   a c u t e   o n s e t   o f   c o u g h ,   s h o n e s s   o f   b r e a t h   a n d   f e w   e p i s o d e s   o f   h e m o p t y s i s .   O n   e x a m i n a t i o n ,   s c l e r a l   i c t e r u s   w a s   p r e s e n t .   T h e r e   i s   a   h i s t o r y   o f   m i n o r   l o w e r   l i m b   i n j u r y   w h i l e   w o r k i n g   i n   t h e   f i e l d s ,   w h i c h   w e r e   i n f e s t e d   w i t h   r a t s .   L a b   f i n d i n g s   r e v e a l e d   a n e m i a ,   l e u k o c y t o s i s ,   d e r a n g e d   R F T s ,   p r o l o n g e d   P T   a n d   a P T T ,   i n c r e a s e d   s e r u m   b i l i r u b i n   a n d   a l k a l i n e   p h o s p h a t a s e   l e v e l s .   B l o o d   c u l 

In [21]:
eval_prompt = """这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。

### Question：
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### output:
"""

In [22]:
# Apply the accelerator. You can comment this out to remove the accelerator.
from accelerate import Accelerator
accelerator = Accelerator()

model = accelerator.prepare_model(model)

In [23]:
# Re-init the tokenizer so it doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    use_fast=False, # needed for now, should be fixed soon
)

In [24]:
device = "cuda"
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to(device)

In [25]:
model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=128)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。

### Question：
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### output:
这个孩子的病状与夏季湿热有关，但是这个孩子的病状也没有关系，因为夏季湿热可能会导致涉及头部的感染，但是这个孩子的病状也�


# 6. Set Up LoRA

In [26]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [27]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (dense): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear8bitLt(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear8bitLt(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_

### **在这里，我们定义了 LoRA 配置。**

- **r** *是低秩矩阵在适配器中使用的秩，它控制了训练的参数数量。较高的秩将允许更高的表达能力，但也会带来计算上的权衡。*:

- **alpha** 是学习到的权重的缩放因子。权重矩阵的缩放因子是 `alpha/r`，因此较高的 alpha 值会赋予 LoRA 激活更多的权重。

在 QLoRA 论文中使用的值是 `r=64` 和 `lora_alpha=16`，这些值被认为能够很好地泛化，但我们将使用 `r=8` 和 `lora_alpha=16`，以便在减少计算复杂度的同时，更加突出新微调数据的重要性。

In [28]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "Wqkv",
        "fc1",
        "fc2",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

trainable params: 6553600 || all params: 2786237440 || trainable%: 0.23521326308787235


In [29]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
              (k_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
              (v_proj): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
              (dense): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
            )
            (mlp): PhiMLP(
              (activation_fn): NewGELUActivation()
              (fc1): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2560, out_features=10240, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  

**7. Run Training!**

我只用了 500 步来快速展示这个笔记本（大约花了一个小时），但我发现模型应该训练得更久，因为在那时它还没有收敛，所以我将步数增加到了 1000 步。甚至可能需要更长的训练时间。

关于训练的注意事项：你可以一开始将 max_steps 设置得比较高，然后检查模型的性能在哪一步开始下降。在这个步骤，你将找到一个适合的训练步数。例如，假设你从 1000 步开始，发现大约 500 步时模型开始过拟合——验证集的损失上升（不理想），而训练集的损失显著下降，这意味着模型学得很好，但无法很好地泛化到新的数据点。因此，500 步是你的最佳步数，你可以在输出目录中使用 checkpoint-500 模型作为最终模型（在下面的第 8 步中）。

当你意识到不再需要训练时，可以通过 Kernel -> Interrupt Kernel 来中断训练过程。

In [30]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [32]:
import transformers
from datetime import datetime

project = "viggo-finetune"
base_model_name = "phi2"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=500,
        learning_rate=2.5e-5,
        logging_steps=25,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
50,1.772500,1.690690
100,1.722000,1.668796
150,1.702700,1.646513
200,1.663900,1.632331
250,1.631900,1.623604
300,1.667200,1.617694
350,1.683400,1.614244
400,1.644100,1.611093
450,1.605600,1.610403
500,1.651500,1.610303


TrainOutput(global_step=500, training_loss=1.6732275924682618, metrics={'train_runtime': 3175.8287, 'train_samples_per_second': 0.63, 'train_steps_per_second': 0.157, 'total_flos': 3.24992249856e+16, 'train_loss': 1.6732275924682618, 'epoch': 0.9857072449482503})